# Scipy

## Table of contents

- [References](#References)
- [Functions are building blocks](#Functions-are-building-blocks)
- [Best practices](#Best-practices)
- [Anatomy of a function](#Anatomy-of-a-function)
    - [Signature](#Signature)
        - [Type hints](#Type-hints)
    - [Body](#Body)
    - [Docstrings](#Docstrings)
- [Parameters and arguments](#Parameters-and-arguments)
    - [Parameters](#Parameters)
    - [Arguments](#Arguments)
    - [Positional arguments](#Positional-arguments)
    - [Keyword arguments](#Keyword-arguments)
    - [Default values](#Default-values)
- [How Python executes a function](#How-Python-executes-a-function)
- [The scope of a function](#The-scope-of-a-function)
- [`*args` and `**kwargs`](#*args-and-**kwargs)
- [Exercises](#Exercises)
    - [Quiz on functions](#Quiz-on-functions)
    - [Longest consecutive sequence 🌶️🌶️](#Longest-consecutive-sequence-🌶️🌶️)
      - [Part 2 🌶️🌶️🌶️](#Part-2-🌶️🌶️🌶️)
    - [Password validator](#Password-validator)
      - [Part 1 🌶️](#Part-1-🌶️)
      - [Part 2 🌶️🌶️](#Part-2-🌶️🌶️)
    - [Buckets reorganization](#Buckets-reorganization)
      - [Part 1 🌶️](#Part-1-🌶️)
      - [Part 2 🌶️🌶️](#Part-2-🌶️🌶️)


## References

Additional material for learning scipy, from the documentation, articles, videos or examples:
- The scipy [user guide](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide).
- A good wikipedia article on [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization).
- The scipy [cookbook](https://scipy-cookbook.readthedocs.io/index.html) with several user-contributed examples of uses for scipy.
- A good [video explanation](https://www.youtube.com/watch?v=Glp7THUpGow) of k-d trees


---

## Introduction: scipy is a mixed bag

Scipy is a python module that provides implementation of many algorithms commonly used in scientific computing. These covers many topics, including:

- Optimization
- Numerical integration
- Linear algebra
- Fourier transforms
- Spatial data processing
- Sparse matrices

And many more. 
Because of the wide variety of algorithms, we will structure this chapter as a series of use cases where the use of scipy would make it easier to solve the problem at hand.
If you want to discover more algorithms offered by scipy, please check the [documentation](https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide).


## Pre-requisites

Most of the algorithms in scipy operate on (multidimensional) arrays of numerical values. 
For technical reasons, these arrays aren't implemented with the built-in python list datatype but using [**numpy arrays**](https://numpy.org/doc/stable/user/absolute_beginners.html). 

These objects offer better memory and computation performance when operating on large arrays as compared to python lists. 
If you want to understand this chapter in-depth and be able to solve the exercises, we advice you first read the chapter on numpy here.



## 1. Use case: optimization

A common problem in the analysis of scientific data is **optimization**.  
In this context, we mean a restricted definition, where given a function and a set of data we want to determine the value of one or more parameters in order to minimize the distance between the function and the data.

### Minimizing the value of a function
Let's see an example to understand what we mean: given a quadratic function `f(x)`, we want to determine the value of `x` that minimizes its value.
We start by defining our function `f`:

In [ ]:
def f(x):
    """
    This is the function we want to minimise
    """
    return x**2 + 4*x + 4 


Now we plot the function using `matplotlib`, a module for plotting described in another module of our course.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Generate x values for plotting
x_values = np.linspace(-10, 2, 100)

# Plot the quadratic function
plt.plot(x_values, f(x_values), label='Quadratic Function')
plt.title('Quadratic Function')

Visually (and algebraically) we can determine the minimum of this function to be at -2. 

We now want to determine the minimum of this function **numerically** using the algorithms offered by scipy. 
Naturally, we would not do this for a quadratic functions, where we can determine the derivatives in a closed form and hence find the exact value of the minimum.
However, the quadratic function is just an example, in reality we would often encounter functions that are too complex to handle algebraically or where the derivate does not exist in a closed form. 
In those cases, we can only determine minima (or solve other optimization problems) using numerical approximation with algorithms like the one scipy offers.

Fortunately, scipy offers the `minimize` [function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html), which can directly estimate the minimum of a function of one argument.
The function takes the function to minimize as a first argument and an initial guess of the minimum as a second argument.

The choice of the initial guess is very important for the success of the minimization: if the function is not **convex**, a wrong choice of initial value will not lead to the true minimum but will result in the algorithm being stuck in a local minimum. 
It is not the purpose of this course to discuss these topics in-detail, if you need more information please consult a book on numerical methods or on optimization.



In [ ]:
from scipy.optimize import minimize
initial_guess = 0  # Initial guess for the minimum
result = minimize(f, initial_guess)
print(result)

As you can see from the output, running `minimze` results in an object with several attributes. 
Of interest for us are primarily the following:

- `success` is a boolean flag which is `True` if the algorithm could find a (local) minimum
- `fun` is the value of the function at the location of the estimated minimum
- `x` is the estimated value of `x` that minimizes the function.

In this case, the algorithm correctly found the minimum of `f` at -2.

### Curve fitting
A closely related problem is **curve fitting**; here given a vector of locations $\mathbf{x} =  \begin{bmatrix} f(x_1) & f(x_2) & \ldots & f(x_n) \end{bmatrix}$, a vector of  observations $\mathbf{y}$ and a function $f(x, \mathbf{k})$ of a location and a set of parameters $\mathbf{k}$  we look for the values of $\mathbf{k}$ that minimize (usually the Euclidian) distance between the points of the function $f$ evaluated at the locations $\mathbf{x}$: $f(\mathbf{x}, \mathbf{k}) = \mathbf{\hat{y}}  = \begin{bmatrix} f(x_1) & f(x_2) & \ldots & f(x_n) \end{bmatrix}$ and the vector of observations $\mathbf{y}$:

$\underset{\mathbf{k}}{\text{min}} \, \| f(\mathbf{x}, \mathbf{k}) - \mathbf{y} \|_2^2$

This means that we look for a set of parameters $\mathbf{k}$ so that the squared differences between the observed values and the values *simulated* by the function are minimized. 
Curve fitting is useful for example when you have a set of experimental data and you want to use them to determine parameters of a physical model you designed your experiment for.


#### 1. Example: determining the gravity acceleration
Suppose for example we want to determine the gravitational acceleration $g$ with an experiment.
To do so, we time the fall of an object from an height $h$ until it touches the ground.  This results in a measurement $t_f$ (the total fall time). In order to obtain enough data, we repeat the experiment for a set of heights $h$.

This defines the following relationship:

$h(t, g) = 1/2 * g t^2$

Because we measure $t$ and vary $h$, we express the problem as:

$t(h, g) = \sqrt{2 h / g}$

Since we can't perform the experiment ourselves, let's first define a function `simulate_fall` to generate the fall time as a function of $h$:


In [ ]:
from numpy.typing import ArrayLike
from numpy.random import randn
def simulate_fall(h: ArrayLike) -> ArrayLike:
    """
    Simulate the fall of an object from a height h
    """
    g = 9.81  # Acceleration due to gravity
    return np.sqrt(2 * h / g) + 1e-3 * randn(len(h))




Notice that we add a random number sampled from a Gaussian distribution with zero mean and standard deviation of 1e-4 seconds to the simulated values. 
We do this to make the measurement more realistic by simulating the effect of measurement noise and imprecisions.
Now we generate 20 fall experiments with heights equally spaced between 0 and 1 m and plot the graph of times against the starting height. 
To make our experiment more statistically representative, we repeat it three times


In [ ]:
n_rep = 4
height_steps = np.linspace(0, 1, 20)
heights = height_steps.repeat(n_rep)
fall_times = simulate_fall(heights)


# Plot the fall times against the heights
fig, ax = plt.subplots()
ax.scatter(heights, fall_times, label='Simulated fall times', s=2)
ax.set_xlabel('Height (m)')
ax.set_ylabel('Fall time (s)')
ax.set_title('Simulated fall times against height')
ax.legend()

At this point we have all the data we need for estimating $g$ from our measurements. 
In this case, the array `fall_times` plays the role of $\mathbf{y}$ and the array `heights` corresponds to $\mathbf{x}$.
We can use the function [`scipy.optimize.curve_fit`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) to solve these types of problems.
The function  takes the following arguments:

- `f`, the function to fit. Its first argument must be `x`, the other arguments correspond to the curve parameters in order of listing
- `xdata` corresponds to the vector $x$
- `ydata` corresponds to the vector $y$


Following this, let's define our function to fit, `f`

In [ ]:
def f(h: ArrayLike, g: float) -> ArrayLike:
    return np.sqrt(2 * h / g)



Now we can call the function:


In [ ]:
from scipy.optimize import curve_fit

# Fit the function to the data
g_est, pcov = curve_fit(f, heights, fall_times)
print(g_est)


Success! As you can see, the estimated value of $g$ is quite close to the real value. The discrepancy is in the order of magnitude of the standard deviation of the noise we add to the simulated measurements.

We can now plot the fitted curve and superimpose it to the "measurements":

In [ ]:
# Plot the fall times against the heights
estimated_fall_times = f(heights, g_est[0])
fig, ax = plt.subplots()
ax.scatter(heights, fall_times, s=2, label='Simulated fall times')
ax.plot(heights, estimated_fall_times, label='Estimated fall times', color='orange')
ax.set_xlabel('Height (m)')
ax.set_ylabel('Fall time (s)')
ax.legend()


You can see that the curve with the estimated fall times given the estimated $g$ tracks the "measurements" very well.
With this example, we conclude the first use case.

## 2. Use case: spatial data
Another place where scipy shines is the processing of **spatial data**. 
This rather generic terms is used to describe data on a plane that represent the locations and shapes of objects, for example maps.
Scipy offers some useful functions to perform basic processing on this data, although for more complex cases there are specific libraries to manipulate and process geospatial data such as [GDAL](https://gdal.org/) that support advanced features like conversion between different geographical coordinate systems, conversion of file format and much more.

### 1. Example: nearest neighbor search
This use case is inspired by a real question a researcher at Empa asked us. 
They approached us for help in analyzing microscopy imaging: they wanted to determine the distribution of the relative orientation angle of bacteria as a function of their distance. This means performing the following steps:

1. Segment an image and extract the outline of each bacteria. 
2. From the outline, we can estimate the direction of the bacteria with respect to the image axis as well as the center point of the bacteria
3. For each bacteria, iterate over all other bacteria forming pairs (bacteria1, bacteria2)
4. For each pair (bacteria1, bacteria2) compute the relative orientation and the distance and store it


The user quickly realized that for a large number of cells in an image this algorithm becomes prohibitively slow to compute. 
Because we perform a pairwise comparison of points, the time (the number of steps) needed to compute the the relative distances and orientations grows with the square of the number of cells. 
However, the user knew that they didn't want to compute **all** distances and orientations but just those **smaller than a certain threshold**.
Thanks to this knowledge and the use of scipy, we could find a method to make their computation much faster.

To do this, we used a special data structure called a [**k-d Tree**](https://en.wikipedia.org/wiki/K-d_tree); this data structure represents a set of k-dimensional points as a tree where each node of the tree splits the space in exactly two halves along one of the coordinates. Thanks to this property, we can very quickly find the nearest points to a given point because at every step in the tree we directly eliminate half of the space to search in. For a better visualization of this idea, please check this [video](https://www.youtube.com/watch?v=Glp7THUpGow).


Let's see how to use this structure with a simplified example. 
To begin with, we create a set of points in the plane, to each of the point we attach a number between 0 and $2\pi$ that represent the orientation of the bacteria with respect to the coordinate system.




In [ ]:
from numpy.typing import ArrayLike
import matplotlib.pyplot as plt
import gstools as gs

def generate_random_field(n: int) -> ArrayLike:
    # structured field with a size 100x100 and a grid-size of 1x1
    x = y = range(100)
    model = gs.Gaussian(dim=2, var=1, len_scale=10)
    srf = gs.SRF(model)
    srf((x, y), mesh_type='structured')
    
   
generate_random_field(1)



### 2. Example: triangulation
Another example of the use of scipy for spatial data comes from the experience of one of us (S. Baffelli). In my past experience as a postdoctoral researcher, I was trying to design an atmospheric sensor network for $CO_2$ observation. The sensors were supposed to be installed at cell sites provided by Swisscom, one of the major Swiss telecom providers. While Swisscom had more than 60 sites in Zürich, only around 20 sensors were available. The goal was then to select cell sites in order to obtain a spatially uniform coverage of the city with sensors.
In other words, the task was as follows:

- Given a list of $n$ coordinates $(x, y)$, select $k$ points out of $n$ so that the area covered by each sensor is balanced: we don't want large gaps in coverage and we don't want some sensors to cover too big of an area.
- 

To do so, we want to satisfy the **max-min coverage conditions**:
*maximize the minimum coverage of the sensors over the area of interest*

More precisely:

- let the variable $x_i$ be a binary variable with $x_i = 1$ if we select the i-th site and $x_i=0$ if the site is not selected. 

Our goal is to find an assignment vector $\hat{\mathbf{x}}$ that satisfies the following conditions


- **Maximize the coverage**: $\text{max}~\underset{\mathbf{i}}{\text{min}} \, C_i$, where $C_i$ is the coverage of the $i$-th assignment
- **Satisfy**: $\sum\limits_{i=0}^{k-1}x_i = k$: we can only assign $k$ sites out of $n$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from numpy.typing import ArrayLike

def generate_points(n, seed=None) -> ArrayLike:
    """Generate n random points within a unit square."""
    np.random.seed(seed)
    points = np.random.rand(n, 2)
    return points

def compute_coverage_areas(points, selected_indices):
    """Compute the coverage areas for selected sensors."""
    selected_points = points[selected_indices, :]
    vor = Voronoi(selected_points)
    areas = []

    for region_index in vor.regions:
        if -1 not in region_index and len(region_index) > 0:
            polygon = vor.vertices[region_index]
            area = 0.5 * np.abs(np.dot(polygon[:, 0], np.roll(polygon[:, 1], 1)) - np.dot(polygon[:, 1], np.roll(polygon[:, 0], 1)))
            areas.append(area)

    return np.array(areas)

def constraint_satisfaction(selected_indices, k):
    """Estimate the satisfaction of the constraint (k sensors selected)."""
    return len(selected_indices) == k

def cost_function(points: ArrayLike, selected_indices: ArrayLike) -> float:
    print(selected_indices)
    int_indices = [int(i) for i in selected_indices]
    """Estimate the cost function based on the sum of squared coverage areas."""
    areas = compute_coverage_areas(points, int_indices)
    #Penalise the cost function if the constraint is not satisfied
    return np.sum(np.square(areas)) + constraint_satisfaction(int_indices, k=5) * 1e6



def display_points(points, selected_points=None):
    """Display the points and optionally highlight selected points."""
    plt.figure(figsize=(8, 8))
    
    # Plot all points
    plt.scatter(points[:, 0], points[:, 1], c='blue', label='All Points')
    
    # Highlight selected points if provided
    if selected_points is not None:
        selected_points = np.array(selected_points)
        plt.scatter(selected_points[:, 0], selected_points[:, 1], c='red', label='Selected Points')
    
    plt.title('Sensor Network Design')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.legend()
    plt.grid(True)
    plt.show()


def get_areas(points: ArrayLike) -> ArrayLike:
    """Get a list of areas for each point."""
    vor = Voronoi(points)
    areas = []

    for region_index in vor.regions:
        if -1 not in region_index and len(region_index) > 0:
            polygon = vor.vertices[region_index]
            area = 0.5 * np.abs(np.dot(polygon[:, 0], np.roll(polygon[:, 1], 1)) - np.dot(polygon[:, 1], np.roll(polygon[:, 0], 1)))
            areas.append(area)

    return np.array(areas)

def estimate_solution(points: ArrayLike, k: int) -> list[int]:
    """Estimate the solution to the sensor network design problem."""
    # Select some initial points
    initial_guess = np.random.choice(len(points), k, replace=False)
    areas = get_areas(points[initial_guess, :])
    cost = cost_function(points, initial_guess)
    print(f"The initial guess is {initial_guess} with cost {cost}")
n_points = 20
points = generate_points(n_points, seed=42)
estimate_solution(points, k=5)  


## 3. Use case: linear algebra 
Another typical use case for scipy is **linear algebra**. 
Here we don't cover all the basics of linear algebra and we don't try to be exhaustive in our coverage;  instead we will show an example that is commonly encountered in a similar form by many scientists: solving a system of linear equations.

### Example: solving a simple linear system
Consider the system of two linear equations:

$$
\begin{align*}
2x + y &= 5 \\
-4x + 6y &= 2
\end{align*}
$$

As we learn in linear algebra, we can express these system as a matrix-vector product:

$\mathbf{A}\mathbf{x} = \mathbf{b}$

Where:
$$
\mathbf{A} = \begin{bmatrix} 2 & 1 \\ -4 & 6 \end{bmatrix}, \quad
\mathbf{x} = \begin{bmatrix} x \\ y \end{bmatrix}, \quad
\mathbf{b} = \begin{bmatrix} 5 \\ 2 \end{bmatrix}
$$

we know that we can solve this system by Gaussian elimination. 
This would be fine for such a small system, however as the number of equations grow, the time it takes to manipulate the matrix $\mathbf{A}$ makes it impossible for a person to find a solution manually.
Fortunately, Gaussian elimination is a well-known algorithm and efficient implementations are available for most programming languages. 
One such implementation can be accessed by using [`scipy.lingalg.solve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve.html).
We now try to solve the system using this function; as a first step we need to express the various matrices and vectors as numpy arrays.


In [ ]:
import numpy as np
import scipy.linalg
A = np.array([[2, 1], [4, 6]])
b = np.array([5 ,2])
x_hat = scipy.linalg.solve(A, b)
print(f"The solution is {x}")

We now can verify if the solution is correct by computing $\mathbf{A}\hat{\mathbf{x}}$, where $\hat{\mathbf{x}}$ is the solution obtained using `solve`. 
When we use numpy arrays, we use the `@` operator to express the matrix-vector product:

In [ ]:
A @ x_hat

Indeed, we obtain the vector that (visually) looks like $\mathbf{b}$.
To verify if indeed the solutions are close, we can look at the distance between $\mathbf{A}\hat{\mathbf{x}}$ und $\mathbf{b}$ using the `scipy.linalg.norm` function:

In [ ]:
scipy.linalg.norm(A @ x_hat - b)

## 4. Use case: interpolation

## 5. Use case: sparse matrices

<div class="alert alert-block alert-info">
    <h4><b>Hint</b></h4> Your solution function takes **only one argument**: a file containing your buckets list. The buckets list is a multi-line string as in the example above. Each line is a single bucket, so you need to split the string on every new line (`\n`)
</div>

In [ ]:
%%ipytest

def solution_buckets1(buckets_file: pathlib.Path) -> int:
    buckets = buckets_file.read_text()  # do NOT remove this line
    # Write your solution here or below this line
    pass